## Linear Regression Final Project: Logistic Regression with Traffic Collision Data
### Zachary Barnes and Bing Wang

##### Housekeeping

In [25]:
# Load Python libraries
import os
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
import matplotlib.pyplot as plt
from scipy.stats import chi2, chi2_contingency

# Run R code adjacent to Python code
%load_ext rpy2.ipython

# Load ggplot R library
%R library(ggplot2)
%R library(scales)

# Avoid kernal death
os.environ['KMP_DUPLICATE_LIB_OK']='True'

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


# Read in and organize data from TIMS (Collisions and Victims)

#### Collisions data

In [2]:
c = pd.read_csv("Collisions.csv")

Response variable: Collision severity. We will code this as a binary variable, with 1 = fatality and 0 = not a fatality

Keep a subset of predictors as full model: Based on previous knowledge, these are likely to be predictors of collision severity

In [3]:
c = c[['COLLISION_TIME','INTERSECTION','COLLISION_SEVERITY',
       'ROAD_SURFACE', 'ROAD_COND_1', 'PEDESTRIAN_ACCIDENT',
       'BICYCLE_ACCIDENT', 'MOTORCYCLE_ACCIDENT', 'ALCOHOL_INVOLVED']]

Make datetime variables

In [4]:
# Convert COLLISION_TIME to hour of day (use tlater to match speeds to collisions)
c['COLLISION_TIME'] = [int(i[:-2]) if len(i) > 2 else 0 for i in c['COLLISION_TIME'].astype(str).values]

c.sample(5)

,COLLISION_TIME,INTERSECTION,COLLISION_SEVERITY,ROAD_SURFACE,ROAD_COND_1,PEDESTRIAN_ACCIDENT,BICYCLE_ACCIDENT,MOTORCYCLE_ACCIDENT,ALCOHOL_INVOLVED
1838,19,Y,4,A,H,Y,NaN,NaN,NaN
1150,20,N,3,A,H,Y,NaN,NaN,NaN
1950,6,Y,3,A,H,NaN,NaN,NaN,Y
2028,18,N,4,A,H,NaN,NaN,NaN,NaN
2348,16,N,3,A,H,Y,NaN,Y,NaN


In [5]:
# recode variables in Collision (as 1s and 0s, reduce categories down, make dummies)

# NOTE: ROAD_SURFACE and ROAD_COND_1 had some observations unstated
# If not stated, assumed no issues with road surface or con'd

# Intersection: Make dummy, intersection/not
c.loc[~(c.INTERSECTION == "Y"), "INTERSECTION"] = 0
c.loc[c.INTERSECTION == "Y", "INTERSECTION"] = 1


# Road_Surface: Convert to dummy, wet/not 
c.loc[~(c.ROAD_SURFACE.isin(["B", "C", "D"])), "WET_ROAD_SURFACE"] = 0
c.loc[c.ROAD_SURFACE.isin(["B", "C", "D"]), "WET_ROAD_SURFACE"] = 1


# Road_Cond_1: Convert to dummy, issue/not
c.loc[~(c.ROAD_COND_1.isin(["H", ""])), "ROAD_COND_ISSUE"] = 1
c.loc[c.ROAD_COND_1.isin(["H", ""]), "ROAD_COND_ISSUE"] = 0

Recode some variables

In [6]:
# for dummies: recode Y as 1, blank as 0
def Yfor1(s):
    s = s.replace("Y", 1)
    s = s.fillna(0)
    return s

In [7]:
c["PEDESTRIAN_ACCIDENT"] = Yfor1(c.PEDESTRIAN_ACCIDENT)
c["BICYCLE_ACCIDENT"] = Yfor1(c.BICYCLE_ACCIDENT)
c["MOTORCYCLE_ACCIDENT"] = Yfor1(c.MOTORCYCLE_ACCIDENT)
c["ALCOHOL_INVOLVED"] = Yfor1(c.ALCOHOL_INVOLVED)

Make response binary variable from Collision_Severity: 1 for fatality (COLLISION_SEVERITY = 1), 0 for not fatality (COLLISION_SEVERITY != 1)

In [8]:
c.loc[c.COLLISION_SEVERITY == 1, "Fatality"] = 1
c.loc[c.COLLISION_SEVERITY != 1, "Fatality"] = 0

Drop vars we won't use

In [9]:
c = c[['Fatality', 'COLLISION_TIME','INTERSECTION', 'PEDESTRIAN_ACCIDENT','BICYCLE_ACCIDENT',
       'MOTORCYCLE_ACCIDENT','WET_ROAD_SURFACE','ROAD_COND_ISSUE']]

In [10]:
c.sample(5)

,Fatality,COLLISION_TIME,INTERSECTION,PEDESTRIAN_ACCIDENT,BICYCLE_ACCIDENT,MOTORCYCLE_ACCIDENT,WET_ROAD_SURFACE,ROAD_COND_ISSUE
1623,0.0,9,1,0.0,0.0,1.0,0.0,1.0
1622,0.0,9,1,0.0,0.0,1.0,0.0,0.0
1395,0.0,13,1,1.0,0.0,0.0,0.0,0.0
635,0.0,16,1,0.0,0.0,0.0,0.0,0.0
1043,0.0,15,1,0.0,0.0,0.0,0.0,0.0


In [ ]:
m =  pd.get_dummies(c, columns = ['PCF_VIOL_CATEGORY'])


## Model Selection 

In [17]:
def forward_selected(data, response, aic_or_bic):
    """Logistic model designed by forward selection.

    Parameters:
    -----------
    data : pandas DataFrame with all possible predictors and response

    response: string, name of response column in data

    Returns:
    --------
    model: an "optimal" fitted statsmodels linear model
           with an intercept
           selected by forward selection
           evaluated by AIC or BIC
    """
    remaining = set(data.columns) 
    remaining.remove(response)
    selected = []
    current_score, best_new_score = 9999, 9999  # set starting point scores really high: want low AIC
    
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        # regress response ~ prior selected predictors + each remaining predictor individually
        for candidate in remaining:
            formula = "{} ~ {} + 1".format(response,
                                           ' + '.join(selected + [candidate]))
            # do aic or bic score, depending on if you specified aic or bic in args
            if aic_or_bic == "AIC":
                score = smf.glm(formula=formula, data=data, family=sm.families.Binomial()).fit().aic 
            elif aic_or_bic == "BIC":
                score = smf.glm(formula=formula, data=data, family=sm.families.Binomial()).fit().bic
            else:
                raise ValueError("Did not specify aic_or_bic = 'AIC' or 'BIC' in args")
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort(reverse=True)
        best_new_score, best_candidate = scores_with_candidates.pop()
        if current_score > best_new_score:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
    
    # final selected model from stepwise selection
    formula = "{} ~ {} + 1".format(response,' + '.join(selected))
    model = smf.glm(formula=formula, data=data, family=sm.families.Binomial()).fit()
    return model

Model chosen by best AIC:

In [18]:
model_aic = forward_selected(c,'Fatality', "AIC")

In [19]:
model_aic.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:               Fatality   No. Observations:                 3870
Model:                            GLM   Df Residuals:                     3866
Model Family:                Binomial   Df Model:                            3
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -132.35
Date:                Sat, 12 Oct 2019   Deviance:                       264.71
Time:                        10:44:58   Pearson chi2:                 3.12e+03
No. Iterations:                     9   Covariance Type:             nonrobust
=======================================================================================
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              -5.2741      0.323    -16.312      0.000      -5.908      -4.640
PEDESTRIAN_ACCIDENT     1.6459      0.416      3.956      0.000       0.830       2.461
INTERSECTION           -1.5631      0.509     -3.072      0.002      -2.560      -0.566
ROAD_COND_ISSUE         1.2175      0.560      2.176      0.030       0.121       2.314
=======================================================================================
"""

Model selected by best BIC:

In [20]:
model_bic = forward_selected(c,'Fatality', "BIC")

In [ ]:
c.INTERSECTION.value_counts()

Unfortunatly, using AIC and BIC does not produce any predictors.

In [21]:
model_bic.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:               Fatality   No. Observations:                 3870
Model:                            GLM   Df Residuals:                     3867
Model Family:                Binomial   Df Model:                            2
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -134.18
Date:                Sat, 12 Oct 2019   Deviance:                       268.37
Time:                        10:45:12   Pearson chi2:                 3.50e+03
No. Iterations:                     9   Covariance Type:             nonrobust
=======================================================================================
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              -5.1657      0.312    -16.532      0.000      -5.778      -4.553
PEDESTRIAN_ACCIDENT     1.6662      0.416      4.008      0.000       0.851       2.481
INTERSECTION           -1.5367      0.508     -3.022      0.003      -2.533      -0.540
=======================================================================================
"""

AIC chose model Fatality ~ PEDESTRIAN_ACCIDENT + INTERSECTION + ROAD_COND_ISSUE

BIC chose model Fatality ~ PEDESTRIAN_ACCIDENT + INTERSECTION 

(makes sense: AIC and BIC work similarly but BIC penalizes model complexity/number of predictors more)

In this model selection, AIC selected a model with a subset of predictors of BIC's model. Let BIC-selected model = full model, and AIC-selected model = reduced model going forward.

In [ ]:
c.head()

AIC and BIC picked similar best models; given BIC 

**Wald test:** 
<br>Check p-values in regression summary. For both full and reduced models, all predictors are significant by Wald test (p-values < alpha = 0.05)

**Deviance test:** 
<br>Let's check whether we should do reduced model or full model.

H0: Reduced model 
<br>Ha: Full model
<br>$\Delta G^{2}$ = $G^{2}$(reduced model) − $G^{2}$(full model)

Compare against $\chi^{2}$($\alpha$ = 0.05, df = $p_{full}$ - $p_{reduced}$ = 3-2 = 1)

In [23]:
delta_G2 = model_bic.deviance - model_aic.deviance
delta_G2

3.6576426516627976

In [26]:
chi2.isf(0.05, 1)

3.8414588206941285

$\Delta G^{2}$ < $\chi^{2}$($\alpha$ = 0.05, df = 1): Fail to reject H0. 
<br>Thus, keep reduced model: Fatality ~ PEDESTRIAN_ACCIDENT + INTERSECTION

### Model selected: Fatality ~ PEDESTRIAN_ACCIDENT + INTERSECTION

# Model Evaluation

Pearson residuals plot